In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import json # library to handle JSON files

import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors # Matplotlib and associated plotting modules

from sklearn.cluster import KMeans # import k-means from clustering stage

!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup # website scraping libraries and packages in Python from BeautifulSoup 

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim  # convert an address into latitude and longitude values

print("Libraries imported.")

Solving environment: / 
  - anaconda/osx-64::ca-certificates-2020.6.24-0, anaconda/osx-64::certifi-2020.6.20-py38_0, anaconda/osx-64::openssl-1.1.1g-h1de35cc_0
  - anaconda/osx-64::ca-certificates-2020.6.24-0, anaconda/osx-64::certifi-2020.6.20-py38_0, defaults/osx-64::openssl-1.1.1g-h1de35cc_0
  - anaconda/osx-64::ca-certificates-2020.6.24-0, anaconda/osx-64::openssl-1.1.1g-h1de35cc_0, defaults/osx-64::certifi-2020.6.20-py38_0
  - anaconda/osx-64::ca-certificates-2020.6.24-0, defaults/osx-64::certifi-2020.6.20-py38_0, defaults/osx-64::openssl-1.1.1g-h1de35cc_0
  - anaconda/osx-64::openssl-1.1.1g-h1de35cc_0, defaults/osx-64::ca-certificates-2020.6.24-0, defaults/osx-64::certifi-2020.6.20-py38_0
  - defaults/osx-64::ca-certificates-2020.6.24-0, defaults/osx-64::certifi-2020.6.20-py38_0, defaults/osx-64::openssl-1.1.1g-h1de35cc_0
  - anaconda/osx-64::certifi-2020.6.20-py38_0, anaconda/osx-64::openssl-1.1.1g-h1de35cc_0, defaults/osx-64::ca-certificates-2020.6.24-0
  - anaconda/osx-64::cer

In [2]:


from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


# 1. Dataset

In [3]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [4]:
soup = BeautifulSoup(data, 'html.parser')

In [5]:
postalCodeList = []
boroughList = []
neighborhoodList = []

In [6]:
soup.find('table').find_all('tr')

# find all the rows of the table
soup.find('table').find_all('tr')

# for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')

In [7]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n'))

## 2. Pandas Dataframe

In [8]:
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned
1,M2A\n,Not assigned\n,Not assigned
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"


In [9]:
toronto_df_drop = toronto_df[toronto_df.Borough != "Not assigned\n"].reset_index(drop=True)
toronto_df_drop.head()

,PostalCode,Borough,Neighborhood
0,M3A\n,North York\n,Parkwoods
1,M4A\n,North York\n,Victoria Village
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights"
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government"


In [10]:
toronto_df_drop = toronto_df_drop.replace('\n','', regex=True)
toronto_df_drop.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [11]:
toronto_df_grouped = toronto_df_drop.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [12]:
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [13]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9V", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford, Maryvale"
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


In [14]:
import numpy as np
import pandas as pd
import json
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup

Solving environment: | 
  - anaconda/osx-64::ca-certificates-2020.6.24-0, anaconda/osx-64::certifi-2020.6.20-py38_0, anaconda/osx-64::openssl-1.1.1g-h1de35cc_0
  - anaconda/osx-64::ca-certificates-2020.6.24-0, anaconda/osx-64::certifi-2020.6.20-py38_0, defaults/osx-64::openssl-1.1.1g-h1de35cc_0
  - anaconda/osx-64::ca-certificates-2020.6.24-0, anaconda/osx-64::openssl-1.1.1g-h1de35cc_0, defaults/osx-64::certifi-2020.6.20-py38_0
  - anaconda/osx-64::ca-certificates-2020.6.24-0, defaults/osx-64::certifi-2020.6.20-py38_0, defaults/osx-64::openssl-1.1.1g-h1de35cc_0
  - anaconda/osx-64::openssl-1.1.1g-h1de35cc_0, defaults/osx-64::ca-certificates-2020.6.24-0, defaults/osx-64::certifi-2020.6.20-py38_0
  - defaults/osx-64::ca-certificates-2020.6.24-0, defaults/osx-64::certifi-2020.6.20-py38_0, defaults/osx-64::openssl-1.1.1g-h1de35cc_0
  - anaconda/osx-64::certifi-2020.6.20-py38_0, anaconda/osx-64::openssl-1.1.1g-h1de35cc_0, defaults/osx-64::ca-certificates-2020.6.24-0
  - anaconda/osx-64::cer

In [15]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

Solving environment: - 
  - anaconda/osx-64::ca-certificates-2020.6.24-0, anaconda/osx-64::certifi-2020.6.20-py38_0, anaconda/osx-64::openssl-1.1.1g-h1de35cc_0
  - anaconda/osx-64::ca-certificates-2020.6.24-0, anaconda/osx-64::certifi-2020.6.20-py38_0, defaults/osx-64::openssl-1.1.1g-h1de35cc_0
  - anaconda/osx-64::ca-certificates-2020.6.24-0, anaconda/osx-64::openssl-1.1.1g-h1de35cc_0, defaults/osx-64::certifi-2020.6.20-py38_0
  - anaconda/osx-64::ca-certificates-2020.6.24-0, defaults/osx-64::certifi-2020.6.20-py38_0, defaults/osx-64::openssl-1.1.1g-h1de35cc_0
  - anaconda/osx-64::openssl-1.1.1g-h1de35cc_0, defaults/osx-64::ca-certificates-2020.6.24-0, defaults/osx-64::certifi-2020.6.20-py38_0
  - defaults/osx-64::ca-certificates-2020.6.24-0, defaults/osx-64::certifi-2020.6.20-py38_0, defaults/osx-64::openssl-1.1.1g-h1de35cc_0
  - anaconda/osx-64::certifi-2020.6.20-py38_0, anaconda/osx-64::openssl-1.1.1g-h1de35cc_0, defaults/osx-64::ca-certificates-2020.6.24-0
  - anaconda/osx-64::cer

In [16]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ 
  - anaconda/osx-64::ca-certificates-2020.6.24-0, anaconda/osx-64::certifi-2020.6.20-py38_0, anaconda/osx-64::openssl-1.1.1g-h1de35cc_0
  - anaconda/osx-64::ca-certificates-2020.6.24-0, anaconda/osx-64::certifi-2020.6.20-py38_0, defaults/osx-64::openssl-1.1.1g-h1de35cc_0
  - anaconda/osx-64::ca-certificates-2020.6.24-0, anaconda/osx-64::openssl-1.1.1g-h1de35cc_0, defaults/osx-64::certifi-2020.6.20-py38_0
  - anaconda/osx-64::ca-certificates-2020.6.24-0, defaults/osx-64::certifi-2020.6.20-py38_0, defaults/osx-64::openssl-1.1.1g-h1de35cc_0
  - anaconda/osx-64::openssl-1.1.1g-h1de35cc_0, defaults/osx-64::ca-certificates-2020.6.24-0, defaults/osx-64::certifi-2020.6.20-py38_0
  - defaults/osx-64::ca-certificates-2020.6.24-0, defaults/osx-64::certifi-2020.6.20-py38_0, defaults/osx-64::openssl-1.1.1g-h1de35cc_0
  - anaconda/osx-64::certifi-2020.6.20-py38_0, anaconda/osx-64::openssl-1.1

In [17]:
toronto_df_grouped.shape

(103, 3)

In [18]:
coordinates = pd.read_csv('https://cocl.us/Geospatial_data')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
toronto_df_new = toronto_df_grouped.merge(coordinates, on="PostalCode", how="left")
toronto_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [21]:
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_new[toronto_df_new["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420


## Borough with toronto

In [22]:
borough_names = list(toronto_df_new.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

## Merging Data

In [23]:
toronto_df_new = toronto_df_new[toronto_df_new['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(toronto_df_new.shape)
toronto_df_new.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [24]:
CLIENT_ID = 'ZYLC4Q3I000O4R32DVJWJJTOTHCGC4O02TXYEPLDAS211SPQ' # your Foursquare ID
CLIENT_SECRET = 'OPKHF1MTRWKRHVR2DAV0IT1IK2H2XZDXJYTCNHVY5L44T55H'  # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZYLC4Q3I000O4R32DVJWJJTOTHCGC4O02TXYEPLDAS211SPQ
CLIENT_SECRET:OPKHF1MTRWKRHVR2DAV0IT1IK2H2XZDXJYTCNHVY5L44T55H


In [25]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['PostalCode'], toronto_df_new['Borough'], 
                                                  toronto_df_new['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id=ZYLC4Q3I000O4R32DVJWJJTOTHCGC4O02TXYEPLDAS211SPQ&client_secret=OPKHF1MTRWKRHVR2DAV0IT1IK2H2XZDXJYTCNHVY5L44T55H&v=20180605 \
     &ll=43.653963,-79.387207&radius=500&limit=100".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

## Convert Venue list to new dataframe

In [26]:
venues_df = pd.DataFrame(venues)


venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2106, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Downtown Toronto,43.653232,-79.385296,Neighborhood
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,Japango,43.655268,-79.385165,Sushi Restaurant
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Sansotei Ramen 三草亭,43.655157,-79.386501,Ramen Restaurant
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Textile Museum of Canada,43.654396,-79.386500,Art Museum
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,Poke Guys,43.654895,-79.385052,Poke Place


In [27]:
venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,54,54,54,54,54,54
M4K,East Toronto,"The Danforth West, Riverdale",54,54,54,54,54,54
M4L,East Toronto,"India Bazaar, The Beaches West",54,54,54,54,54,54
M4M,East Toronto,Studio District,54,54,54,54,54,54
M4N,Central Toronto,Lawrence Park,54,54,54,54,54,54
M4P,Central Toronto,Davisville North,54,54,54,54,54,54
M4R,Central Toronto,"North Toronto West, Lawrence Park",54,54,54,54,54,54
M4S,Central Toronto,Davisville,54,54,54,54,54,54
M4T,Central Toronto,"Moore Park, Summerhill East",54,54,54,54,54,54


In [28]:
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(2106, 44)


,PostalCode,Borough,Neighborhoods,Art Gallery,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop,Café,...,Pub,Ramen Restaurant,Salon / Barbershop,Seafood Restaurant,Smoke Shop,Sushi Restaurant,Tea Room,Toy / Game Store,University,Vegetarian / Vegan Restaurant
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,M4E,East Toronto,The Beaches,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Group rows by neighborhood and taking the mean of each category

In [29]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(39, 44)


,PostalCode,Borough,Neighborhoods,Art Gallery,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop,Café,...,Pub,Ramen Restaurant,Salon / Barbershop,Seafood Restaurant,Smoke Shop,Sushi Restaurant,Tea Room,Toy / Game Store,University,Vegetarian / Vegan Restaurant
0,M4E,East Toronto,The Beaches,0.055556,0.018519,0.018519,0.037037,0.037037,0.018519,0.055556,...,0.018519,0.018519,0.018519,0.018519,0.018519,0.037037,0.018519,0.018519,0.018519,0.018519
1,M4K,East Toronto,"The Danforth West, Riverdale",0.055556,0.018519,0.018519,0.037037,0.037037,0.018519,0.055556,...,0.018519,0.018519,0.018519,0.018519,0.018519,0.037037,0.018519,0.018519,0.018519,0.018519
2,M4L,East Toronto,"India Bazaar, The Beaches West",0.055556,0.018519,0.018519,0.037037,0.037037,0.018519,0.055556,...,0.018519,0.018519,0.018519,0.018519,0.018519,0.037037,0.018519,0.018519,0.018519,0.018519
3,M4M,East Toronto,Studio District,0.055556,0.018519,0.018519,0.037037,0.037037,0.018519,0.055556,...,0.018519,0.018519,0.018519,0.018519,0.018519,0.037037,0.018519,0.018519,0.018519,0.018519
4,M4N,Central Toronto,Lawrence Park,0.055556,0.018519,0.018519,0.037037,0.037037,0.018519,0.055556,...,0.018519,0.018519,0.018519,0.018519,0.018519,0.037037,0.018519,0.018519,0.018519,0.018519
5,M4P,Central Toronto,Davisville North,0.055556,0.018519,0.018519,0.037037,0.037037,0.018519,0.055556,...,0.018519,0.018519,0.018519,0.018519,0.018519,0.037037,0.018519,0.018519,0.018519,0.018519
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",0.055556,0.018519,0.018519,0.037037,0.037037,0.018519,0.055556,...,0.018519,0.018519,0.018519,0.018519,0.018519,0.037037,0.018519,0.018519,0.018519,0.018519
7,M4S,Central Toronto,Davisville,0.055556,0.018519,0.018519,0.037037,0.037037,0.018519,0.055556,...,0.018519,0.018519,0.018519,0.018519,0.018519,0.037037,0.018519,0.018519,0.018519,0.018519
8,M4T,Central Toronto,"Moore Park, Summerhill East",0.055556,0.018519,0.018519,0.037037,0.037037,0.018519,0.055556,...,0.018519,0.018519,0.018519,0.018519,0.018519,0.037037,0.018519,0.018519,0.018519,0.018519
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",0.055556,0.018519,0.018519,0.037037,0.037037,0.018519,0.055556,...,0.018519,0.018519,0.018519,0.018519,0.018519,0.037037,0.018519,0.018519,0.018519,0.018519


## New dataframe to display the top 10 venues for each PostalCode

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(39, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym
1,M4K,East Toronto,"The Danforth West, Riverdale",Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym
2,M4L,East Toronto,"India Bazaar, The Beaches West",Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym
3,M4M,East Toronto,Studio District,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym
4,M4N,Central Toronto,Lawrence Park,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym
5,M4P,Central Toronto,Davisville North,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym
7,M4S,Central Toronto,Davisville,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym
8,M4T,Central Toronto,"Moore Park, Summerhill East",Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym


# Clustering 

In [31]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

<ipython-input-31-e6b3e1ad4717>:6: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [32]:
#create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_merged = toronto_df_new.copy()

# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")

print(toronto_merged.shape)
toronto_merged.head() # check the last columns!

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym


In [33]:
# sort the results by Cluster Labels
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym
21,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym
22,M5N,Central Toronto,Roselawn,43.711695,-79.416936,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym
23,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym
24,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym
25,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym
26,M5T,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym
27,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym
20,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym
28,M5W,Downtown Toronto,Stn A PO Boxes,43.646435,-79.374846,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym


## Check

### Cluster 0

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + \
                                                                                 list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym
21,Downtown Toronto,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym
22,Central Toronto,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym
23,Central Toronto,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym
24,Central Toronto,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym
25,Downtown Toronto,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym
26,Downtown Toronto,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym
27,Downtown Toronto,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym
20,Downtown Toronto,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym
28,Downtown Toronto,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Hotel,Sushi Restaurant,Bar,Breakfast Spot,Food Court,Gym


### Cluster 1

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + \
                                                                                 list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


### Cluster 2

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + \
                                                                                 list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


### Cluster 3


In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + \
                                                                                 list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


### Cluster 4

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + \
                                                                                 list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


# Conclution 

Majority of the neighborhoods fall into Cluster 0 which consists of cafes and restaurants.

# Part 2

In [40]:
import requests
import pandas as pd
import numpy as np

In [41]:
conda install -c anaconda xlrd --yes

Solving environment: done

## Package Plan ##

  environment location: /Users/vishnu/opt/anaconda3

  added / updated specs:
    - xlrd


The following packages will be SUPERSEDED by a higher-priority channel:

  conda              conda-forge::conda-4.8.5-py38h32f6830~ --> anaconda::conda-4.8.5-py38_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [42]:
conda install -c anaconda lxml --yes

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [43]:
!pip install folium
import folium

In [44]:
url  = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
if page.status_code == 200:
    print('Page download successful')
else:
    print('Page download error. Error code: {}'.format(page.status_code))

Page download successful


In [45]:
import lxml
df_html = pd.read_html(url, header=0, na_values = ['Not assigned'])[0]
df_html.head()

,Postal Code,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [46]:
df_html.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
df_html.head()

,PostalCode,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [47]:
df_html.dropna(subset=['Borough'], inplace=True)

In [48]:
n_empty_neighborhood = df_html[df_html['Neighbourhood'].isna()].shape[0]
print('Number of rows on which Neighborhood column is empty: {}'.format(n_empty_neighborhood))

Number of rows on which Neighborhood column is empty: 0


In [49]:
df_html[df_html['Neighbourhood'].isna()]

,PostalCode,Borough,Neighbourhood


In [50]:
df_html['Neighbourhood'].fillna(df_html['Borough'], inplace=True)
n_empty_neighborhood = df_html[df_html['Neighbourhood'].isna()].shape[0]
print('Number of rows on which Neighborhood column is empty: {}'.format(n_empty_neighborhood))

Number of rows on which Neighborhood column is empty: 0


In [51]:
df_html[df_html['Borough']=="Queen's Park"]

,PostalCode,Borough,Neighbourhood


In [52]:
df_postalcodes =pd.DataFrame( df_html.groupby(['PostalCode','Borough']).Neighbourhood.agg([('Neighbourhood', ', '.join)]))
df_postalcodes.reset_index(inplace=True)
df_postalcodes.head(5)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [53]:
df_html.dropna(subset=['Borough'], inplace=True)

In [54]:
n_empty_neighborhood = df_html[df_html['Neighbourhood'].isna()].shape[0]
print('Number of rows on which Neighborhood column is empty: {}'.format(n_empty_neighborhood))



Number of rows on which Neighborhood column is empty: 0


In [55]:
print('The shape of the dataset is:',df_postalcodes.shape)



The shape of the dataset is: (103, 3)


In [56]:
df_postalcodes.to_csv('Toronto_Postcodes.csv')

In [57]:
url_csv = 'http://cocl.us/Geospatial_data'
df_coordinates = pd.read_csv(url_csv)
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [58]:
df_neighborhoods = pd.read_csv('Toronto_Postcodes.csv',index_col=[0])
df_neighborhoods.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [59]:
df_coordinates.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df_neighborhoods.rename(columns={'Postcode': 'PostalCode'}, inplace=True)



In [60]:
df_neighborhoods_coordinates = pd.merge(df_neighborhoods, df_coordinates, on='PostalCode')
df_neighborhoods_coordinates.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [61]:
df_neighborhoods_coordinates[(df_neighborhoods_coordinates['PostalCode']=='M5G') |
                             (df_neighborhoods_coordinates['PostalCode']=='M2H') ]

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
17,M2H,North York,Hillcrest Village,43.803762,-79.363452
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [62]:
df_neighborhoods_coordinates.to_csv('Toronto_Postcodes_2.csv')

In [63]:
df = pd.read_csv('Toronto_Postcodes_2.csv', index_col=0)
df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [64]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [65]:
df.rename(columns={'Neighbourhood': 'Neighborhood'}, inplace=True)

In [66]:
df.groupby('Borough').count()['Neighborhood']

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Scarborough         17
West Toronto         6
York                 5
Name: Neighborhood, dtype: int64

In [67]:
df_toronto = df[df['Borough'].str.contains('Toronto')]
df_toronto.reset_index(inplace=True)
df_toronto.drop('index', axis=1, inplace=True)
df_toronto.head()

/Users/vishnu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [68]:
print(df_toronto.groupby('Borough').count()['Neighborhood'])



Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
West Toronto         6
Name: Neighborhood, dtype: int64


In [69]:
#Create list with the Boroughs (to be used later)
boroughs = df_toronto['Borough'].unique().tolist()

In [70]:
lat_toronto = df_toronto['Latitude'].mean()
lon_toronto = df_toronto['Longitude'].mean()
print('The geographical coordinates of Toronto are {}, {}'.format(lat_toronto, lon_toronto))



The geographical coordinates of Toronto are 43.66713498717948, -79.38987324871795


In [71]:
borough_color = {}
for borough in boroughs:
    borough_color[borough]= '#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3))

In [72]:
map_toronto = folium.Map(location=[lat_toronto, lon_toronto], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], 
                                           df_toronto['Longitude'],
                                           df_toronto['Borough'], 
                                           df_toronto['Neighborhood']):
    label_text = borough + ' - ' + neighborhood
    label = folium.Popup(label_text)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=borough_color[borough],
        fill_color=borough_color[borough],
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [73]:
CLIENT_ID = 'ZYLC4Q3I000O4R32DVJWJJTOTHCGC4O02TXYEPLDAS211SPQ' # your Foursquare ID
CLIENT_SECRET = 'OPKHF1MTRWKRHVR2DAV0IT1IK2H2XZDXJYTCNHVY5L44T55H' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [74]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
   
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [75]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                latitudes=df_toronto['Latitude'],
                                longitudes=df_toronto['Longitude'])

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

In [76]:
toronto_venues.shape

(1634, 7)

In [77]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,55,55,55,55,55,55
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,71,71,71,71,71,71
Christie,16,16,16,16,16,16
Church and Wellesley,77,77,77,77,77,77
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33


In [78]:
toronto_venues['Venue Category'].unique()[:100]

array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood',
       'Greek Restaurant', 'Cosmetics Shop', 'Ice Cream Shop',
       'Italian Restaurant', 'Brewery', 'Juice Bar', 'Yoga Studio',
       'Fruit & Vegetable Store', 'Pizza Place', 'Restaurant',
       'Dessert Shop', 'Bookstore', 'Furniture / Home Store', 'Spa',
       'Bubble Tea Shop', 'Café', 'Caribbean Restaurant', 'Grocery Store',
       'Coffee Shop', 'Bakery', 'Indian Restaurant',
       'American Restaurant', 'Lounge', 'Frozen Yogurt Shop',
       'Liquor Store', 'Sushi Restaurant', 'Gym', 'Fish & Chips Shop',
       'Fast Food Restaurant', 'Park', 'Pet Store', 'Steakhouse',
       'Burrito Place', 'Movie Theater', 'Sandwich Place', 'Fish Market',
       'Gay Bar', 'Seafood Restaurant', 'Cheese Shop',
       'Middle Eastern Restaurant', 'Stationery Store',
       'Comfort Food Restaurant', 'Wine Bar', 'Thai Restaurant',
       'Coworking Space', 'Latin American Restaurant', 'Bar', 'Gastropub',
       'Gym / Fitness Cen

In [79]:
"Indian Restaurant" in toronto_venues['Venue Category'].unique()

True

In [80]:
to_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Neighborhoods'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

print(to_onehot.shape)
to_onehot.head()

(1634, 239)


,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [81]:
to_grouped = to_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(to_grouped.shape)
to_grouped

(39, 239)


,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.018182,0.000000,0.000000,0.000000,0.00,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.0625,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.014085,0.000000,0.000000,0.014085,0.00,0.014085
5,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,Church and Wellesley,0.012987,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.012987,0.000000,...,0.012987,0.000000,0.000000,0.00,0.000000,0.000000,0.012987,0.000000,0.00,0.025974
7,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.040000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.00,0.000000
8,Davisville,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.030303,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,Davisville North,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000


In [82]:
len(to_grouped[to_grouped["Indian Restaurant"] > 0])

7

In [83]:
to_indian = to_grouped[["Neighborhoods","Indian Restaurant"]]
to_indian.head(9)

,Neighborhoods,Indian Restaurant
0,Berczy Park,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000
2,"Business reply mail Processing Centre, South C...",0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000
4,Central Bay Street,0.014085
5,Christie,0.000000
6,Church and Wellesley,0.012987
7,"Commerce Court, Victoria Hotel",0.000000
8,Davisville,0.030303


In [84]:
from sklearn.cluster import KMeans
toclusters = 3

to_clustering = to_indian.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=1)
kmeans.fit_transform(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]

array([1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1],
      dtype=int32)

In [85]:
to_merged = to_indian.copy()

# add clustering labels
to_merged["Cluster Labels"] = kmeans.labels_

In [86]:
to_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
to_merged.head(5)

,Neighborhood,Indian Restaurant,Cluster Labels
0,Berczy Park,0.000000,1
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,1
2,"Business reply mail Processing Centre, South C...",0.000000,1
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,1
4,Central Bay Street,0.014085,0


In [87]:
to_merged = to_merged.join(toronto_venues.set_index("Neighborhood"), on="Neighborhood")

print(to_merged.shape)
to_merged.head()

(1634, 9)


,Neighborhood,Indian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.0,1,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
0,Berczy Park,0.0,1,43.644771,-79.373306,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant
0,Berczy Park,0.0,1,43.644771,-79.373306,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
0,Berczy Park,0.0,1,43.644771,-79.373306,Meridian Hall,43.646292,-79.376022,Concert Hall
0,Berczy Park,0.0,1,43.644771,-79.373306,Goose Island Brewhouse,43.647329,-79.373541,Beer Bar


In [88]:
to_merged.sort_values(["Cluster Labels"], inplace=True)
to_merged.head()

,Neighborhood,Indian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
36,"The Danforth West, Riverdale",0.023256,0,43.679557,-79.352188,IQ Living,43.678477,-79.347811,Furniture / Home Store
6,Church and Wellesley,0.012987,0,43.665860,-79.383160,Asahi Sushi,43.669874,-79.382943,Sushi Restaurant
6,Church and Wellesley,0.012987,0,43.665860,-79.383160,Alexander Wood Statue,43.663702,-79.380301,Sculpture Garden
6,Church and Wellesley,0.012987,0,43.665860,-79.383160,Noah's Natural Foods,43.668532,-79.385885,Food & Drink Shop
6,Church and Wellesley,0.012987,0,43.665860,-79.383160,Flash,43.664319,-79.380190,Strip Club


In [89]:
map_clusters = folium.Map(location=[lat_toronto, lon_toronto],zoom_start=14)

# set color scheme for the clusters


# add markers to the map
markers_colors={}
markers_colors[0] = 'red'
markers_colors[1] = 'blue'
markers_colors[2] = 'green'
markers_colors[3] = 'yellow'
markers_colors[4] = 'cyan'
markers_colors[5] = 'black'
for lat, lon, cluster in zip(to_merged['Neighborhood Latitude'], to_merged['Neighborhood Longitude'], to_merged['Cluster Labels']):
    
    
    folium.CircleMarker(
        [lat, lon],
        radius=5,
       
        color =markers_colors[cluster],
        fill_color=markers_colors[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [90]:
#Cluster 0
to_merged.loc[(to_merged['Cluster Labels'] ==0) & (to_merged['Venue Category'] == 'Indian Restaurant') ]

,Neighborhood,Indian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
36,"The Danforth West, Riverdale",0.023256,0,43.679557,-79.352188,Sher-E-Punjab,43.677308,-79.353066,Indian Restaurant
6,Church and Wellesley,0.012987,0,43.665860,-79.383160,Kothur Indian Cuisine,43.667872,-79.385659,Indian Restaurant
8,Davisville,0.030303,0,43.704324,-79.388790,Marigold Indian Bistro,43.702881,-79.388008,Indian Restaurant
14,"Harbourfront East, Union Station, Toronto Islands",0.010000,0,43.640816,-79.381752,Indian Roti House,43.639060,-79.385422,Indian Restaurant
30,"St. James Town, Cabbagetown",0.020408,0,43.667967,-79.367675,Butter Chicken Factory,43.667072,-79.369184,Indian Restaurant
4,Central Bay Street,0.014085,0,43.657952,-79.387383,Colaba Junction,43.660940,-79.385635,Indian Restaurant


In [91]:
#Cluster 1
to_merged.loc[(to_merged['Cluster Labels'] ==1) & (to_merged['Venue Category'] == 'Indian Restaurant') ]

,Neighborhood,Indian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


In [92]:
#Cluster 2

to_merged.loc[(to_merged['Cluster Labels'] ==2) & (to_merged['Venue Category'] == 'Indian Restaurant') ]

,Neighborhood,Indian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
34,"The Annex, North Midtown, Yorkville",0.052632,2,43.67271,-79.405678,Roti Cuisine of India,43.674618,-79.408249,Indian Restaurant


In [93]:
#Cluster 3

to_merged.loc[(to_merged['Cluster Labels'] ==3) & (to_merged['Venue Category'] == 'Indian Restaurant') ]

,Neighborhood,Indian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


In [94]:
#Cluster 4

to_merged.loc[(to_merged['Cluster Labels'] ==4) & (to_merged['Venue Category'] == 'Indian Restaurant') ]

,Neighborhood,Indian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


# Observation

Most of the Indian restaurants are in cluster 0 which is around Davisville, Church and Wellesley, Central Bay Street etc.
There are a low number of restaurants in Cluster 2 areas which is in The Annex, North Midtown, Yorkville. Also, there are good opportunities to open restaurant near St James Town, Cabbagetown, Regent Park, The Annex, North Midtown, Yorkville.


# Conclusion 

It is observed that cluster 2 might be a good location as there are a limited number of Indian restaurants in these areas. Therefore, it is recommended to open an Indian restaurant in these locations.